<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FINAL_NVIDIA_ai_virtual_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/tutorials/quickstart/quickstart.ipynb

In [1]:
!git clone https://github.com/NVIDIA-AI-Blueprints/ai-virtual-assistant.git

Cloning into 'ai-virtual-assistant'...
remote: Enumerating objects: 543, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 543 (delta 128), reused 77 (delta 77), pack-reused 320 (from 2)
Receiving objects: 100% (543/543), 2.08 MiB | 27.25 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [2]:
from google.colab import userdata
import os
api_key = userdata.get('NVIDIA_API_KEY')
import os
os.environ['NVIDIA_API_KEY'] = api_key

In [3]:
%cd /content/ai-virtual-assistant

/content/ai-virtual-assistant


Installing Native Postgres and Redis

In [4]:
import os, subprocess, time
from google.colab import userdata

# 1. INSTALL POSTGRES & REDIS NATIVELY (Bypass Udocker for Infra)
print("🚀 Installing Native Postgres and Redis...")
!apt-get update -qq
!apt-get install -y postgresql redis-server -qq
!service postgresql start
!service redis-server start

# Configure Native Postgres
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'password';"
!sudo -u postgres p

🚀 Installing Native Postgres and Redis...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Preconfiguring packages ...
Selecting previously unselected package libjemalloc2:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../00-libjemalloc2_5.2.1-4ubuntu1_amd64.deb ...
Unpacking libjemalloc2:amd64 (5.2.1-4ubuntu1) ...
Selecting previously unselected package liblua5.1-0:amd64.
Preparing to unpack .../01-liblua5.1-0_5.1.5-8.1build4_amd64.deb ...
Unpacking liblua5.1-0:amd64 (5.1.5-8.1build4) ...
Selecting previously unselected package liblzf1:amd64.
Preparing to unpack .../02-liblzf1_3.6-3_amd64.deb ...
Unpacking liblzf1:amd64 (3.6-3) ...
Selecting previously unselected package lua-bitop:amd64.
Preparing to unpack .../03-lua-bitop_1.0.2-5_amd64.deb ...
Unpacking lua-bitop:amd64 (1.0.2-5) ...
Selecting

 Ensuring Postgres and Redis are running

netstat -punta |egrep '5432|6379'

In [5]:
import os
import time
import subprocess
import sys

# 1. CLEANUP & INSTALL
print("🧹 Clearing port 19530...")
!fuser -k 19530/tcp || true
!pip install milvus-lite -q

# 2. START NATIVE SERVICES (Postgres and Redis)
print("🚀 Ensuring Postgres and Redis are running...")
!service postgresql start
!service redis-server start

# 3. START MILVUS SERVER (The Python-Native Way)
print("🔥 Launching Milvus on Port 19530...")
# This uses the python module runner to ensure the code is found
cmd = f"nohup {sys.executable} -m milvus_lite.server --port 19530 > milvus_final.log 2>&1 &"
os.system(cmd)

print("⏳ Waiting 20s for initialization...")
time.sleep(20)

# 4. FINAL INFRASTRUCTURE VERIFICATION
print("\n--- INFRASTRUCTURE STATUS ---")
!netstat -tulpn | grep -E '5432|6379|19530'

# 5. DIAGNOSIS (If 19530 is still missing)
if os.path.exists('milvus_final.log'):
    with open('milvus_final.log', 'r') as f:
        logs = f.read()
        if "Address already in use" in logs:
            print("⚠️ Port 19530 is already busy. Killing and retrying...")
        elif "No module named" in logs:
             print("❌ Module path error. Trying fallback...")
             os.system(f"nohup {sys.executable} -m milvus_lite --port 19530 > milvus_final.log 2>&1 &")

🧹 Clearing port 19530...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 12.7 MB/s eta 0:00:00
🚀 Ensuring Postgres and Redis are running...
 * Starting PostgreSQL 14 database server
   ...done.
Starting redis-server: redis-server.
🔥 Launching Milvus on Port 19530...
⏳ Waiting 20s for initialization...

--- INFRASTRUCTURE STATUS ---
tcp        0      0 127.0.0.1:5432          0.0.0.0:*               LISTEN      2891/postgres       
tcp        0      0 127.0.0.1:6379          0.0.0.0:*               LISTEN      2927/redis-server 1 
tcp6       0      0 ::1:5432                :::*                    LISTEN      2891/postgres       
tcp6       0      0 ::1:6379                :::*                    LISTEN      2927/redis-server 1 


Installing Milvus components

In [6]:
# 1. Install necessary libraries
print("🛠️ Installing Milvus components...")
!pip install -U pymilvus milvus-lite -q

# 2. Restart and Configure Environment
import os
from pymilvus import MilvusClient

# Create a local database file (the 'Lite' way)
# This avoids Port 19530 'DOWN' errors entirely
db_file = "./milvus_quickstart.db"
client = MilvusClient(db_file)
collection_name = "quickstart_collection"

# 3. Create Collection
print(f"📦 Creating collection: {collection_name}")
if client.has_collection(collection_name):
    client.drop_collection(collection_name)

client.create_collection(
    collection_name=collection_name,
    dimension=5,  # Tutorial example dimension
)

# 4. Insert Sample Data
print("📥 Inserting data...")
data = [
    {"id": i, "vector": [i * 0.1] * 5, "text": f"Entity {i}"}
    for i in range(10)
]
client.insert(collection_name=collection_name, data=data)

# 5. Search (Verification)
print("🔍 Searching for nearest neighbors...")
res = client.search(
    collection_name=collection_name,
    data=[[0.5] * 5],
    limit=3,
    output_fields=["text"]
)

for result in res[0]:
    print(f"Found: {result['entity']['text']} (Distance: {result['distance']})")

print("\n✅ Quickstart successful using Milvus-Lite!")

🛠️ Installing Milvus components...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 5.0 MB/s eta 0:00:00
📦 Creating collection: quickstart_collection
📥 Inserting data...
🔍 Searching for nearest neighbors...
Found: Entity 3 (Distance: 1.0)
Found: Entity 2 (Distance: 1.0)
Found: Entity 1 (Distance: 1.0)

✅ Quickstart successful using Milvus-Lite!


In [7]:
import os

# 1. Point the Agent to the local .db file instead of a Port
# This is the 'secret sauce' that connects the blueprint to your quickstart
os.environ['APP_VECTORSTORE_URL'] = "/content/milvus_quickstart.db"
os.environ['APP_VECTORSTORE_NAME'] = "milvus"
os.environ['COLLECTION_NAME'] = "quickstart_collection"

# 2. Set your NVIDIA API Key (Ensure this is in your Colab Secrets)
from google.colab import userdata
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

print("✅ Agent environment configured for Milvus-Lite.")

✅ Agent environment configured for Milvus-Lite.


Native Infrastructure with Cloud Persistence

In [9]:
import os, subprocess, time, socket
from google.colab import drive

# 1. MOUNT DRIVE & PREP FOLDERS
print("📂 Mounting Drive and Setting up Sync Bridge...")
drive.mount('/content/drive', force_remount=True)
DRIVE_BACKUP = "/content/drive/MyDrive/NVIDIA_AI_ASSISTANT/postgres_backup"
LOCAL_DATA = "/var/lib/postgresql/14/main"
!mkdir -p {DRIVE_BACKUP}
!mkdir -p /var/run/postgresql /content/logs
!chmod 777 /content/logs

# 2. CLEANUP & RESTORE FROM DRIVE (If backup exists)
print("🔄 Restoring data from Google Drive to local storage...")
!pkill -9 postgres || true
!rm -rf {LOCAL_DATA}
!mkdir -p {LOCAL_DATA}
# Copy data from Drive to Local so Postgres can 'own' it
if os.path.exists(f"{DRIVE_BACKUP}/PG_VERSION"):
    !cp -r {DRIVE_BACKUP}/* {LOCAL_DATA}/
else:
    print("✨ No backup found. Starting a fresh cluster.")

# 3. SET PERMISSIONS (Local folders allow this)
!chown -R postgres:postgres /var/lib/postgresql
!chmod 700 {LOCAL_DATA}

# 4. INITIALIZE IF NEW
if not os.path.exists(f"{LOCAL_DATA}/PG_VERSION"):
    !sudo -u postgres /usr/lib/postgresql/14/bin/initdb -D {LOCAL_DATA}

# 5. START POSTGRES
print("🐘 Starting Postgres locally...")
!sudo -u postgres /usr/lib/postgresql/14/bin/pg_ctl \
    -D {LOCAL_DATA} \
    -o "-c config_file=/etc/postgresql/14/main/postgresql.conf -c unix_socket_directories='/var/run/postgresql'" \
    -l /content/logs/postgres_sync.log restart

# 6. START REDIS
print("🔴 Starting Redis...")
!pkill -9 redis-server || true
!nohup redis-server --port 6379 --protected-mode no > /content/logs/redis_sync.log 2>&1 &

# 7. THE PERSISTENCE TRIGGER (Backup back to Drive)
def sync_to_drive():
    print("💾 Syncing local database to Google Drive...")
    !cp -r {LOCAL_DATA}/* {DRIVE_BACKUP}/
    print("✅ Backup complete.")

# 8. VERIFY
time.sleep(10)
print("\n--- 🏥 STABLE PERSISTENT STATUS ---")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(2)
if s.connect_ex(('127.0.0.1', 5432)) == 0:
    print(f"Postgres  : ✅ LIVE (Local + Sync)")
    sync_to_drive() # Initial backup
else:
    print(f"Postgres  : ❌ DOWN. Checking log...")
    !tail -n 5 /content/logs/postgres_sync.log
s.close()

📂 Mounting Drive and Setting up Sync Bridge...
Mounted at /content/drive
🔄 Restoring data from Google Drive to local storage...
🐘 Starting Postgres locally...
pg_ctl: old server process (PID: 5405) seems to be gone
starting server anyway
waiting for server to start.... done
server started
🔴 Starting Redis...

--- 🏥 STABLE PERSISTENT STATUS ---
Postgres  : ✅ LIVE (Local + Sync)
💾 Syncing local database to Google Drive...
✅ Backup complete.


In [10]:
# 1. Install missing LangChain integration packages
print("📦 Patching LangChain integrations...")
!pip install langchain-community langchain-milvus -q

📦 Patching LangChain integrations...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [12]:
!pip install langchain-nvidia-ai-endpoints -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.6 MB/s eta 0:00:00


In [13]:
import os
import gradio as gr
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_milvus import Milvus
from google.colab import userdata

# 2. Setup NVIDIA Models & Milvus-Lite Connection
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
embeddings = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")

# Connect to the .db file you created in the quickstart
vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": "/content/milvus_quickstart.db"},
    collection_name="quickstart_collection",
    auto_id=True
)
retriever = vectorstore.as_retriever()

# 3. Define RAG Logic
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def real_respond(message, history):
    # Retrieve relevant data from Milvus-Lite
    docs = retriever.invoke(message)
    context = "\n".join([doc.page_content for doc in docs])

    # Generate answer using Llama 3.1
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": message})

# 4. Launch the Final UI
print("🚀 Launching the NVIDIA AI Virtual Assistant...")
demo = gr.ChatInterface(
    fn=real_respond,
    type="messages",
    title="NVIDIA AI Virtual Assistant",
    description="Full Stack: Postgres, Redis, and Milvus-Lite"
)
demo.launch(share=True)

🚀 Launching the NVIDIA AI Virtual Assistant...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a89cd5395b66a9cdbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
import os, subprocess, time

# 1. FORCE-CREATE THE POSTGRES USER
print("👤 Creating 'postgres' system user...")
!groupadd -r postgres || true
!useradd -r -g postgres -d /var/lib/postgresql -s /bin/bash postgres || true

# 2. INSTALL & INITIALIZE DATA DIRECTORY
print("🐘 Installing and Initializing Postgres...")
!apt-get install postgresql-14 -y -qq
!mkdir -p /var/lib/postgresql/14/main /var/run/postgresql
!chown -R postgres:postgres /var/lib/postgresql/ /var/run/postgresql/

# If the database isn't initialized, do it now
if not os.path.exists("/var/lib/postgresql/14/main/PG_VERSION"):
    print("✨ Initializing new database cluster...")
    !sudo -u postgres /usr/lib/postgresql/14/bin/initdb -D /var/lib/postgresql/14/main

# 3. DIRECT LAUNCH
print("🚀 Launching services manually...")

# Start Postgres
!sudo -u postgres /usr/lib/postgresql/14/bin/pg_ctl \
    -D /var/lib/postgresql/14/main \
    -l /content/logs/postgres_new.log start

# Start Redis (using the package we installed in the last step)
!nohup redis-server --port 6379 --protected-mode no > /content/logs/redis_new.log 2>&1 &

# 4. FINAL VERIFICATION
time.sleep(10)
print("\n--- 🏥 INFRASTRUCTURE STATUS ---")
import socket
for name, port in [("Postgres", 5432), ("Redis", 6379)]:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(2)
    check = s.connect_ex(('127.0.0.1', port))
    print(f"{name:<10} (Port {port}): {'✅ LIVE' if check == 0 else '❌ STILL DOWN'}")
    s.close()

👤 Creating 'postgres' system user...
groupadd: group 'postgres' already exists
useradd: user 'postgres' already exists
🐘 Installing and Initializing Postgres...
🚀 Launching services manually...
pg_ctl: another server might be running; trying to start server anyway
waiting for server to start.... stopped waiting
pg_ctl: could not start server
Examine the log output.

--- 🏥 INFRASTRUCTURE STATUS ---
Postgres   (Port 5432): ✅ LIVE
Redis      (Port 6379): ✅ LIVE


In [15]:
!pip install langchain-nvidia-ai-endpoints -q

In [16]:
import os
import socket
import sys
from pymilvus import MilvusClient
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from google.colab import userdata

def run_full_health_check():
    # 1. Re-sync Authentication
    os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

    print("🔍 STARTING FULL STACK DIAGNOSTICS...\n")
    results = []

    # --- LAYER 1: NATIVE INFRASTRUCTURE (Postgres & Redis) ---
    infra_ports = {"Postgres": 5432, "Redis": 6379}
    for name, port in infra_ports.items():
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(2)
        is_live = s.connect_ex(('127.0.0.1', port)) == 0
        s.close()
        results.append(is_live)
        print(f"{'✅' if is_live else '❌'} {name:<15} (Port {port}): {'LIVE' if is_live else 'DOWN'}")

    # --- LAYER 2: VECTOR STORAGE (Milvus-Lite) ---
    milvus_uri = "/content/milvus_quickstart.db"
    try:
        client = MilvusClient(milvus_uri)
        count = len(client.list_collections())
        mv_ok = True
        mv_msg = f"LIVE ({count} collections found)"
    except Exception as e:
        mv_ok = False
        mv_msg = f"ERROR: {str(e)}"
    results.append(mv_ok)
    print(f"{'✅' if mv_ok else '❌'} {'Milvus-Lite':<15} (File-Based): {mv_msg}")

    # --- LAYER 3: INTELLIGENCE (NVIDIA NIM) ---
    try:
        # Using the verified stable 8b model for the heartbeat check
        llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
        response = llm.invoke("health_check_ping")
        nim_ok = True
        nim_msg = "CONNECTED (Llama 3.1)"
    except Exception as e:
        nim_ok = False
        nim_msg = f"FAILED: Check API Key or Quota"
    results.append(nim_ok)
    print(f"{'✅' if nim_ok else '❌'} {'NVIDIA NIM':<15} (Cloud API): {nim_msg}")

    # --- FINAL VERDICT ---
    print("\n" + "="*45)
    if all(results):
        print("🚀 SYSTEM STATUS: ALL SYSTEMS OPERATIONAL")
        print("Your Virtual Assistant is ready for use.")
    else:
        print("⚠️  SYSTEM STATUS: INFRASTRUCTURE DEGRADED")
        print("Please re-run the setup cell for the '❌' components.")
    print("="*45)

run_full_health_check()

🔍 STARTING FULL STACK DIAGNOSTICS...

✅ Postgres        (Port 5432): LIVE
✅ Redis           (Port 6379): LIVE
✅ Milvus-Lite     (File-Based): LIVE (0 collections found)
✅ NVIDIA NIM      (Cloud API): CONNECTED (Llama 3.1)

🚀 SYSTEM STATUS: ALL SYSTEMS OPERATIONAL
Your Virtual Assistant is ready for use.


In [ ]:
!pip install pypdf -q

In [18]:
!pip install pymilvus -q
!pip install langchain-milvus -q
!pip install langchain-community -q

In [19]:
import os
import requests
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_milvus import Milvus
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from pymilvus import MilvusClient
from google.colab import userdata

def sanitize_metadata(documents):
    """Clean metadata keys to ensure Milvus compatibility."""
    for doc in documents:
        clean_metadata = {}
        for key, value in doc.metadata.items():
            clean_key = re.sub(r'[^a-zA-Z0-9_]', '_', key)
            if not clean_key[0].isalpha() and clean_key[0] != '_':
                clean_key = "field_" + clean_key
            clean_metadata[clean_key] = value
        doc.metadata = clean_metadata
    return documents

def download_and_ingest(pdf_url, collection_name="quickstart_collection"):
    # 1. DOWNLOAD
    local_filename = "auto_downloaded_doc.pdf"
    print(f"📥 Downloading: {pdf_url}...")
    response = requests.get(pdf_url)
    with open(local_filename, 'wb') as f:
        f.write(response.content)

    # 2. LOAD AND CLEAN
    loader = PyPDFLoader(local_filename)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = sanitize_metadata(text_splitter.split_documents(docs))

    # 3. NVIDIA SETUP
    os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
    embeddings = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")
    milvus_uri = "/content/milvus_quickstart.db"

    # 4. RESET COLLECTION (Fixes the DataNotMatchException)
    client = MilvusClient(milvus_uri)
    if client.has_collection(collection_name):
        print(f"🗑️ Dropping old {collection_name} to fix ID schema...")
        client.drop_collection(collection_name)

    # 5. FRESH INGESTION
    print(f"🧬 Storing {len(splits)} chunks in Milvus-Lite...")
    vectorstore = Milvus.from_documents(
        documents=splits,
        embedding=embeddings,
        connection_args={"uri": milvus_uri},
        collection_name=collection_name,
        drop_old=True,  # This ensures the new auto_id schema is applied
        auto_id=True
    )
    print(f"🚀 SUCCESS! {pdf_url} is now searchable with Auto-ID enabled.")

# Run the fixed pipeline
target_pdf = "https://media.geeksforgeeks.org/wp-content/uploads/20240226121023/GFG.pdf"
download_and_ingest(target_pdf)

📥 Downloading: https://media.geeksforgeeks.org/wp-content/uploads/20240226121023/GFG.pdf...
🧬 Storing 61 chunks in Milvus-Lite...
🚀 SUCCESS! https://media.geeksforgeeks.org/wp-content/uploads/20240226121023/GFG.pdf is now searchable with Auto-ID enabled.


In [20]:
import os
import requests
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_milvus import Milvus
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from pymilvus import MilvusClient
from google.colab import userdata

def sanitize_metadata(documents):
    """Clean metadata keys to ensure Milvus compatibility."""
    for doc in documents:
        clean_metadata = {}
        for key, value in doc.metadata.items():
            # Milvus only allows alphanumeric and underscores for metadata keys
            clean_key = re.sub(r'[^a-zA-Z0-9_]', '_', key)
            if not clean_key[0].isalpha() and clean_key[0] != '_':
                clean_key = "field_" + clean_key
            clean_metadata[clean_key] = value
        doc.metadata = clean_metadata
    return documents

def download_and_ingest(urls, collection_name="ai_agent_knowledge", reset=False):
    # 1. SETUP ENVIRONMENT
    os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
    embeddings = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")
    milvus_uri = "/content/milvus_quickstart.db"

    # Initialize Milvus Client to check for/reset collections
    client = MilvusClient(milvus_uri)
    if reset and client.has_collection(collection_name):
        print(f"🗑️ Resetting collection: {collection_name}")
        client.drop_collection(collection_name)

    all_splits = []

    # 2. PROCESS AI PDFs
    for url in urls:
        try:
            local_filename = f"ai_doc_{urls.index(url)}.pdf"
            print(f"📥 Downloading: {url}...")

            response = requests.get(url, timeout=15)
            with open(local_filename, 'wb') as f:
                f.write(response.content)

            # Load PDF
            loader = PyPDFLoader(local_filename)
            docs = loader.load()

            # Enrich metadata
            for doc in docs:
                doc.metadata["source_url"] = url

            # 3. CONSERVATIVE SPLITTING (Prevents Token Limit Exception)
            # 500 characters is approximately 125-150 tokens, well within the 512 limit.
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=500,
                chunk_overlap=50,
                separators=["\n\n", "\n", ".", " ", ""]
            )

            splits = sanitize_metadata(text_splitter.split_documents(docs))
            all_splits.extend(splits)

            # Clean up temp file
            os.remove(local_filename)

        except Exception as e:
            print(f"❌ Failed to process {url}: {e}")

    # 4. FINAL INGESTION
    if all_splits:
        print(f"🧬 Generating embeddings for {len(all_splits)} chunks...")
        vectorstore = Milvus.from_documents(
            documents=all_splits,
            embedding=embeddings,
            connection_args={"uri": milvus_uri},
            collection_name=collection_name,
            drop_old=reset,
            auto_id=True
        )
        print(f"🚀 SUCCESS! {len(urls)} AI PDFs indexed in Milvus.")
        return vectorstore
    else:
        print("⚠️ No documents were processed.")
        return None

# --- AI TARGET LIST ---

ai_pdf_list = [
    "https://arxiv.org/pdf/2306.02224.pdf",        # Auto-GPT
    "https://arxiv.org/pdf/2003.10217.pdf",        # Milvus
    "https://arxiv.org/pdf/2303.17564.pdf",        # Tool Makers
    "https://arxiv.org/pdf/1404.7828.pdf",         # Schmidhuber DL
    "https://arxiv.org/pdf/2403.17881.pdf",        # Deepfake/Proteomics
    "https://arxiv.org/pdf/2409.03741.pdf",        # ML Attacks
    "https://arxiv.org/pdf/2203.02155.pdf",        # RLHF (InstructGPT)
    "https://arxiv.org/pdf/2305.18290.pdf",        # DPO
    "https://arxiv.org/pdf/1707.06347.pdf",        # PPO
    "https://arxiv.org/pdf/1506.02438.pdf"         # GAE (Generalized Advantage Estimation)
]

# Run the pipeline
# Set reset=True on first run to ensure clean schema
vectorstore = download_and_ingest(ai_pdf_list, reset=True)

📥 Downloading: https://arxiv.org/pdf/2306.02224.pdf...
📥 Downloading: https://arxiv.org/pdf/2003.10217.pdf...
📥 Downloading: https://arxiv.org/pdf/2303.17564.pdf...
📥 Downloading: https://arxiv.org/pdf/1404.7828.pdf...
📥 Downloading: https://arxiv.org/pdf/2403.17881.pdf...
📥 Downloading: https://arxiv.org/pdf/2409.03741.pdf...
📥 Downloading: https://arxiv.org/pdf/2203.02155.pdf...
📥 Downloading: https://arxiv.org/pdf/2305.18290.pdf...
📥 Downloading: https://arxiv.org/pdf/1707.06347.pdf...
📥 Downloading: https://arxiv.org/pdf/1506.02438.pdf...
🧬 Generating embeddings for 2813 chunks...
🚀 SUCCESS! 10 AI PDFs indexed in Milvus.


In [21]:
from pymilvus import MilvusClient
import os

# 1. Connect to the Lite DB
client = MilvusClient("/content/milvus_quickstart.db")
collection_name = "ai_agent_knowledge"

if client.has_collection(collection_name):
    # Get total count
    stats = client.get_collection_stats(collection_name=collection_name)
    total_chunks = stats.get('row_count', 0)
    print(f"📊 Collection: {collection_name} | Total Chunks: {total_chunks}")

    # 2. Get ALL unique sources first
    # We query all rows to ensure we don't miss a document hidden deep in the PKs
    all_data = client.query(
        collection_name=collection_name,
        filter="pk >= 0",
        output_fields=["source_url"],
        limit=total_chunks
    )

    unique_sources = list(set([entry['source_url'] for entry in all_data if 'source_url' in entry]))
    print(f"✅ Found {len(unique_sources)} unique documents in the database.")

    # 3. Peek at ONE chunk per source
    print("\n📑 Representative Samples from Each Source:")
    print("-" * 65)

    for i, source in enumerate(sorted(unique_sources)):
        # Retrieve exactly one chunk for this specific source
        sample = client.query(
            collection_name=collection_name,
            filter=f'source_url == "{source}"',
            limit=1,
            output_fields=["text"]
        )

        if sample:
            content = sample[0].get('text', 'No content').replace('\n', ' ')
            filename = os.path.basename(source)
            print(f"[{i+1}] FILE: {filename}")
            print(f"    SAMPLED CONTENT: {content[:140]}...")
            print("-" * 65)

else:
    print(f"❌ Collection '{collection_name}' not found.")

📊 Collection: ai_agent_knowledge | Total Chunks: 2813
✅ Found 10 unique documents in the database.

📑 Representative Samples from Each Source:
-----------------------------------------------------------------
[1] FILE: 1404.7828.pdf
    SAMPLED CONTENT: Deep Learning in Neural Networks: An Overview Technical Report IDSIA-03-14 / arXiv:1404.7828 v4 [cs.NE] (88 pages, 888 references) J¨urgen S...
-----------------------------------------------------------------
[2] FILE: 1506.02438.pdf
    SAMPLED CONTENT: Published as a conference paper at ICLR 2016 HIGH -D IMENSIONAL CONTINUOUS CONTROL USING GENERALIZED ADVANTAGE ESTIMATION John Schulman, Phi...
-----------------------------------------------------------------
[3] FILE: 1707.06347.pdf
    SAMPLED CONTENT: Proximal Policy Optimization Algorithms John Schulman, Filip Wolski, Prafulla Dhariwal, Alec Radford, Oleg Klimov OpenAI {joschu, filip, pra...
-----------------------------------------------------------------
[4] FILE: 2003.10217.pdf

In [22]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. Initialize the LLM (Llama 3.1 70B)
llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct") # 🚀 Upgraded to 405B

# 2. Create the RAG Prompt
template = """
You are a professional AI Research Assistant. Use the following pieces of retrieved context
to answer the question. If you don't know the answer, just say that you don't know.
Keep the answer concise and professional.

Context:
{context}

Question: {question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. Setup the Retriever from your existing vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 4. Construct the RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- TEST IT ---
user_query = "Compare the performance of GPT-4 vs Claude in Auto-GPT tasks based on the research."
response = rag_chain.invoke(user_query)

print(f"🤖 AI Assistant:\n{response}")

🤖 AI Assistant:
According to the research, GPT-4 demonstrated superior performance in Auto-GPT tasks, achieving a success rate of 0.485 and precision of 0.628, while Claude's performance was markedly inferior, with a success rate of 0.075 or 0.082 in different settings.


Compare the 'Spatio-Temporal Credit Assignment' problem defined by Schmidhuber with the 'Variance-Bias Tradeoff' solved by GAE. How does the latter provide a practical implementation for the former in modern autonomous agents?

In [23]:
import os
import gradio as gr
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_milvus import Milvus
from pymilvus import MilvusClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

# 1. SETUP - Utilizing the 405B parameter model
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct") # 🚀 Upgraded to 405B
embeddings = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")
milvus_uri = "/content/milvus_quickstart.db"
COLLECTION_NAME = "ai_agent_knowledge"

PERSONAS = {
    "AI Research Assistant": "You are a professional AI Research Assistant. Focus on high-fidelity technical data.",
    "Aerospace Engineer": "You are a Senior Aerospace Engineer. Your answers must be grounded in physical and safety constraints.",
    "Medical AI Specialist": "You are a Medical AI Researcher. Focus on clinical evidence and proteomic data accuracy.",
    "AI Portfolio Manager": "You are an AI Portfolio Manager. Evaluate technical readiness and commercial feasibility."
}

def get_available_docs():
    client = MilvusClient(milvus_uri)
    if not client.has_collection(COLLECTION_NAME): return ["No Documents Found"]
    res = client.query(collection_name=COLLECTION_NAME, filter='pk >= 0', output_fields=["source_url"])
    sources = list(set([os.path.basename(r['source_url']) for r in res if 'source_url' in r]))
    return ["All Documents"] + sorted(sources)

# 2. ORCHESTRATION LOGIC (Optimized for 405B reasoning)
def orchestrate_respond(message, history, selected_doc, selected_persona, strict_mode):
    vectorstore = Milvus(
        embedding_function=embeddings,
        connection_args={"uri": milvus_uri},
        collection_name=COLLECTION_NAME
    )

    search_kwargs = {"k": 6}
    if selected_doc != "All Documents":
        search_kwargs["expr"] = f'source_url LIKE "%{selected_doc}"'

    retriever = vectorstore.as_retriever(search_kwargs=search_kwargs)
    docs = retriever.invoke(message)
    context_text = "\n\n".join([f"Source: {os.path.basename(d.metadata.get('source_url', ''))} (Page {d.metadata.get('page', 'N/A')}):\n{d.page_content}" for d in docs])

    system_role = PERSONAS.get(selected_persona, PERSONAS["AI Research Assistant"])

    # 3. ADVANCED STRICT PROMPT (Tailored for 405B)
    if strict_mode:
        instruction = """
        CRITICAL INSTRUCTION (STRICT MODE):
        You are acting as a closed-system retrieval interface.
        1. Access ONLY the provided context to answer.
        2. If the answer is not explicitly found in the text, you MUST state: 'The provided context does not contain the answer.'
        3. DO NOT use your internal training data to supplement the response.
        4. Cite the specific file and page for every factual claim.
        """
    else:
        instruction = "Answer using the context provided, but you may use your internal expertise to explain complex terms found in the text."

    template = f"""{system_role}
    {instruction}

    ### DATA SOURCE (CONTEXT):
    {{context}}

    ### USER INQUIRY:
    {{question}}

    ### RESPONSE:"""

    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": context_text, "question": message})

    citations = list(set([f"📄 {os.path.basename(d.metadata.get('source_url', ''))} (Pg {d.metadata.get('page', 'N/A')})" for d in docs]))
    return f"{answer}\n\n---\n**Audit Trail:**\n{', '.join(citations)}"

# 4. UI CONSTRUCTION
print("🚀 Launching 405B Orchestrator...")
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🏛️ NVIDIA 405B Agentic Orchestrator")
    gr.Markdown("Using Llama-3.1-405B for maximum reasoning capability.")

    with gr.Row():
        doc_dropdown = gr.Dropdown(choices=get_available_docs(), value="All Documents", label="Knowledge Source", scale=2)
        persona_dropdown = gr.Dropdown(choices=list(PERSONAS.keys()), value="AI Research Assistant", label="Persona", scale=2)
        strict_toggle = gr.Checkbox(label="Strict Grounding", value=True, scale=1)
        refresh_btn = gr.Button("🔄", scale=0)

    chat = gr.ChatInterface(
        fn=orchestrate_respond,
        additional_inputs=[doc_dropdown, persona_dropdown, strict_toggle],
        type="messages"
    )

    refresh_btn.click(fn=lambda: gr.update(choices=get_available_docs()), outputs=doc_dropdown)

demo.launch(share=True)

🚀 Launching 405B Orchestrator...


/tmp/ipython-input-1457080810.py:82: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e16a70e8832b69d5d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Compare the 'Spatio-Temporal Credit Assignment' problem defined by Schmidhuber with the 'Variance-Bias Tradeoff' solved by GAE. How does the latter provide a practical implementation for the former in modern autonomous agents?

To compare the 'Spatio-Temporal Credit Assignment' problem defined by Schmidhuber with the 'Variance-Bias Tradeoff' solved by GAE, we must analyze the context provided.

The Spatio-Temporal Credit Assignment problem, as mentioned in Schmidhuber's dissertation (1404.7828.pdf, Page 75), refers to the challenge of assigning credit to internal components of a dynamic system, such as a neural network, for their contributions to the system's overall behavior and performance. This problem is fundamental in understanding complex systems and optimizing their performance.

On the other hand, the Variance-Bias Tradeoff, as described in the context of Generalized Advantage Estimation (GAE) (1506.02438.pdf, Page 10), is a specific problem in reinforcement learning where an agent must balance the tradeoff between the variance of its estimates and the bias of its learning process. GAE provides a practical solution to this problem by combining techniques such as trust region policy optimization and value function optimization, both represented by neural networks.

To relate the two concepts, we can see that the Spatio-Temporal Credit Assignment problem is a more general and abstract problem, while the Variance-Bias Tradeoff is a specific instance of this problem in the context of reinforcement learning. The response function, as defined in (1506.02438.pdf, Page 5), lets us quantify the temporal credit assignment problem, which is a specific aspect of the Spatio-Temporal Credit Assignment problem. The response function decomposes the advantage function across timesteps, allowing us to understand the temporal dependencies between actions and rewards.

In modern autonomous agents, GAE provides a practical implementation for the Spatio-Temporal Credit Assignment problem by offering a method to estimate the advantage function, which is a key component in assigning credit to the agent's actions. By using GAE, agents can learn to solve complex control tasks that have previously been out of reach for generic reinforcement learning methods.

In conclusion, GAE provides a practical solution to the Variance-Bias Tradeoff, which is a specific instance of the more general Spatio-Temporal Credit Assignment problem. By using GAE, modern autonomous agents can better assign credit to their actions and optimize their performance in complex environments.

References:

Schmidhuber, J. (1990a). Dynamische neuronale Netze und das fundamentale raumzeitliche Lern- problem. (Dynamic neural nets and the fundamental spatio-temporal credit assignment problem.) Dissertation, Inst. f. Inf., Tech. Univ. Munich. (1404.7828.pdf, Page 75)
1506.02438.pdf, Page 10
1506.02438.pdf, Page 5
Audit Trail:
📄 1404.7828.pdf (Pg 76), 📄 1506.02438.pdf (Pg 5), 📄 1506.02438.pdf (Pg 10), 📄 1506.02438.pdf (Pg 1), 📄 1404.7828.pdf (Pg 75)



